In [1]:
import numpy as np

In [2]:
# 載入資料
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

In [3]:
# 重塑數據形狀為 (樣本數, 高度, 寬度, 頻道數)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1776, 64, 1034, 1)
(222, 64, 1034, 1)
(222, 64, 1034, 1)


In [4]:
import tensorflow as tf
from keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from tensorflow.keras.layers import BatchNormalization

def create_model(input_shape):
    input_img = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_img, outputs=output)
    return model


In [5]:
input_shape = np.shape(X_train)[1:]
model = create_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 1034, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 64, 1034, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 64, 1034, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 517, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 517, 64)       18496     
                                                                 
 batch_normalization_1 (Batc  (None, 32, 517, 64)      256   

# 編譯模型

In [6]:
from keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[reduce_lr], verbose=2)


Epoch 1/50
56/56 - 7s - loss: 2.5653 - accuracy: 0.5113 - val_loss: 0.6889 - val_accuracy: 0.5450 - lr: 0.0010 - 7s/epoch - 130ms/step
Epoch 2/50
56/56 - 3s - loss: 1.1968 - accuracy: 0.5428 - val_loss: 0.6940 - val_accuracy: 0.5180 - lr: 0.0010 - 3s/epoch - 48ms/step
Epoch 3/50
56/56 - 3s - loss: 0.9094 - accuracy: 0.5152 - val_loss: 0.7000 - val_accuracy: 0.4820 - lr: 0.0010 - 3s/epoch - 48ms/step
Epoch 4/50
56/56 - 3s - loss: 0.7617 - accuracy: 0.5090 - val_loss: 0.6932 - val_accuracy: 0.5180 - lr: 0.0010 - 3s/epoch - 48ms/step
Epoch 5/50
56/56 - 3s - loss: 0.7037 - accuracy: 0.5450 - val_loss: 0.6938 - val_accuracy: 0.5180 - lr: 0.0010 - 3s/epoch - 48ms/step
Epoch 6/50
56/56 - 3s - loss: 0.6901 - accuracy: 0.5715 - val_loss: 0.6960 - val_accuracy: 0.5180 - lr: 0.0010 - 3s/epoch - 48ms/step
Epoch 7/50
56/56 - 3s - loss: 0.6685 - accuracy: 0.5884 - val_loss: 0.6654 - val_accuracy: 0.5315 - lr: 5.0000e-04 - 3s/epoch - 48ms/step
Epoch 8/50
56/56 - 3s - loss: 0.6441 - accuracy: 0.5901 -